<a href="https://colab.research.google.com/github/BrahianCarrera/KaggleCompetition/blob/main/099%20-%20Modelo%20con%20preprocesado%20por%20moda%20y%20catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
# Carga el archivo kaggle.json
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai4eng-20241
!unzip udea-ai4eng-20241.zip

Saving kaggle.json to kaggle.json


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
test_df = pd.read_csv('/content/test.csv', encoding= 'utf-8')
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')

In [ ]:
df

In [ ]:
def Preprocesar(df):
  def obtener_primera_frase(cadena):
    return cadena.split(' ', 1)[0]

  def quitar_tildes(palabra):
    return re.sub(r'[ÁÉÍÓÚ]', lambda x: {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}[x.group()], palabra)

  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(obtener_primera_frase)
  df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(quitar_tildes)

  reemplazos = {
      'ADMINISTRACION': ['ADMINISTRACION', 'ADMINISTRACI¿N', 'ADMINISTRACIÒN', 'ADMINISTRACIÓN', 'ADMINSITRACION', 'ADMINSITRACIÓN'],
      'ANTROPOLOGIA': ['ANTROPOLOGIA', 'ANTROPOLOGÍA'],
      'ARTES': ['ARTE', 'ARTES','BELLAS','ANIMACION'],
      'BIOINGENIERIA': ['BIOINGENIERIA', 'BIOINGENIERÍA'],
      'BIOLOGIA': ['BIOLOGIA', 'BIOLOGÍA'],
      'COMUNICACION': ['COMUNICACION', 'COMUNICACIONES', 'COMUNICACI¿N', 'COMUNICACIÓN', 'COMUNICACIÖN'],
      'CONSTRUCCION': ['CONSTRUCCION', 'CONSTRUCCIONES'],
      'CONTADURIA': ['CONTADURIA', 'CONTADURÍA'],
      'DISEÑO': ['DISENO', 'DISE¿O', 'DISE¿¿O', 'DISEÑO'],
      'ECOLOGIA': ['ECOLOGIA', 'ECOLOGÍA'],
      'ECONOMIA': ['ECONOM¿A'],
      'GEOLOGIA': ['GEOLOG¿A'],
      'GERENCIA': ['GERONTOLOGIA'],
      'INGENIERIA': ['INGENIER¿A', 'INGENIERÌA', 'INGENIIERIA','AGROINDUSTRIA'],
      'INSTRUMENTACION': ['INTRUMENTACION'],
      'MATEMATICAS': ['MATEMATICA'],
      'MARKETING': ['MERCADEO', 'MERCADOLOGIA'],
      'PSICOLOGIA': ['PSICOLOGA'],
      'QUIMICA': ['QUIMICA.', 'QU¿MICA'],
      'MUSICA': ['MUSICA-'],
      'TERAPIAS': ['TERAPIA', 'TERAPIAS'],
      'GESTION': ['GESTI¿N'],
      'NEGOCIOS' : ['3°'],
      'DEPORTE' : ['ACTIVIDAD'],
      'LICENCIATURA' : ['Licenciatura','LICENC.EN'],
      'LENGUAS' :['LENGUAJES']

  }

  for key, values in reemplazos.items():
      df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].replace(values, key)

  ##Eliminar columnas

  df = df.drop(labels =['ESTU_PRIVADO_LIBERTAD','FAMI_TIENEINTERNET.1'],axis= 1)

  if 'RENDIMIENTO_GLOBAL' in df.columns:
    df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE','RENDIMIENTO_GLOBAL']
  else:
    df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE']

  ##Convertir a binario

  def convertir_num(valor):
      if valor == 'Si':
          return 1
      elif valor == 'No':
          return 0
      else:
          return valor

  # Aplicar la función a todas las columnas del DataFrame
  df = df.applymap(convertir_num)

  ##Imputar Datos

  df['ESTRATO'] = df['ESTRATO'].str.replace('Estrato ', '')

  columnas_binarias = ['TIENE_INTERNET', 'TIENE_LAVADORA', 'TIENE_AUTO',
                        'PAGA_MATRICULA', 'TIENE_COMPUTADOR']

  for columna in columnas_binarias:
      moda = df[columna].mode()[0]
      df[columna].fillna(moda, inplace=True)


  ModaEstrato = df['ESTRATO'].mode()[0]  # Obtener la moda
  df['ESTRATO'].fillna(ModaEstrato, inplace=True)

  ModaPadre = df['EDUCACION_PADRE'].mode()[0]
  df['EDUCACION_PADRE'].fillna(ModaPadre, inplace=True)

  ModaMadre = df['EDUCACION_MADRE'].mode()[0]
  df['EDUCACION_MADRE'].fillna(ModaMadre, inplace=True)

  ValorMatricula = df['VALOR_MATRICULA'].mode()[0]
  df['VALOR_MATRICULA'].fillna(ValorMatricula, inplace=True)

  HorasTrabaja = df['HORAS_TRABAJA'].mode()[0]
  df['HORAS_TRABAJA'].fillna(HorasTrabaja, inplace=True)

  ##Convertir columnas categoricas
  Columnas_categoricas = ["VALOR_MATRICULA","PROGRAMA","DEPARTAMENTO","EDUCACION_MADRE","EDUCACION_PADRE"]

  df[Columnas_categoricas] = df[Columnas_categoricas].astype("category")

  valores_rendimiento = {'alto': 4,'medio-alto': 3,'medio-bajo':2, 'bajo': 1}
  df= df.replace(valores_rendimiento)

  ##Convertir a Onehot
  encoder = OneHotEncoder(sparse=False)

  encoded_columns = encoder.fit_transform(df[['HORAS_TRABAJA','PERIODO','VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE']])

  feature_names = encoder.get_feature_names_out(['HORAS_TRABAJA','PERIODO', 'VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'])

  one_hot_encoded_df = pd.DataFrame(encoded_columns, columns=feature_names)

  df = pd.concat([df, one_hot_encoded_df], axis=1)

  df = df.drop(labels =['HORAS_TRABAJA', 'PERIODO','VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'],axis= 1)

  return df

In [ ]:
df = Preprocesar(df)
test_df = Preprocesar(test_df)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
df.describe()

,ID,TIENE_INTERNET,TIENE_LAVADORA,TIENE_AUTO,PAGA_MATRICULA,TIENE_COMPUTADOR,HORAS_TRABAJA_0,HORAS_TRABAJA_Entre 11 y 20 horas,HORAS_TRABAJA_Entre 21 y 30 horas,HORAS_TRABAJA_Menos de 10 horas,...,EDUCACION_MADRE_Ninguno,EDUCACION_MADRE_No Aplica,EDUCACION_MADRE_No sabe,EDUCACION_MADRE_Postgrado,EDUCACION_MADRE_Primaria completa,EDUCACION_MADRE_Primaria incompleta,EDUCACION_MADRE_Secundaria (Bachillerato) completa,EDUCACION_MADRE_Secundaria (Bachillerato) incompleta,EDUCACION_MADRE_Técnica o tecnológica completa,EDUCACION_MADRE_Técnica o tecnológica incompleta
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,...,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,494606.130576,0.894069,0.870994,0.341186,0.438702,0.918084,0.168303,0.167303,0.133853,0.125908,...,0.020914,0.002770,0.004357,0.066781,0.081047,0.143567,0.238856,0.116985,0.129303,0.039759
std,285585.209455,0.307749,0.335207,0.474108,0.496229,0.274237,0.374136,0.373246,0.340494,0.331745,...,0.143097,0.052555,0.065861,0.249643,0.272907,0.350650,0.426385,0.321402,0.335535,0.195392
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,247324.750000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,494564.500000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,741782.500000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,989286.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Crea el mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de calor de correlaciones')
plt.show()

NameError: name 'correlation_matrix' is not defined

<Figure size 800x600 with 0 Axes>

In [ ]:
category_cols = [i for i in df.columns if not i in df._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(df[c].dropna()))

RENDIMIENTO_GLOBAL ['alto' 'bajo' 'medio-alto' 'medio-bajo']


In [ ]:
print(df.isnull().sum())

TIENE_INTERNET                                          0
TIENE_LAVADORA                                          0
TIENE_AUTO                                              0
PAGA_MATRICULA                                          0
TIENE_COMPUTADOR                                        0
                                                       ..
EDUCACION_MADRE_Primaria incompleta                     0
EDUCACION_MADRE_Secundaria (Bachillerato) completa      0
EDUCACION_MADRE_Secundaria (Bachillerato) incompleta    0
EDUCACION_MADRE_Técnica o tecnológica completa          0
EDUCACION_MADRE_Técnica o tecnológica incompleta        0
Length: 198, dtype: int64


In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

X = df.drop(labels =["RENDIMIENTO_GLOBAL",'ID'], axis=1)  # Características
y = df["RENDIMIENTO_GLOBAL"]  # Variable objetivo

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Crear el modelo CatBoost
catboost_model = CatBoostClassifier(iterations=2500, learning_rate=0.1, depth=6, loss_function='MultiClass')

# Entrenar el modelo
catboost_model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = catboost_model.score(X_test, y_test)
print("Accuracy:", accuracy)

In [ ]:
category_cols = [i for i in test_df.columns if not i in test_df._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(df[c].dropna()))

In [ ]:
predictions = catboost_model.predict(test_df.drop('ID', axis=1))

predictions = predictions.flatten()

submit = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predictions})

submit['RENDIMIENTO_GLOBAL'] = submit['RENDIMIENTO_GLOBAL'].replace({
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
})

submit.to_csv('submission.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

# Preprocesamiento de datos
X = df.drop("RENDIMIENTO_GLOBAL", axis=1)  # Características
y = df["RENDIMIENTO_GLOBAL"]  # Variable objetivo

# Codificar las etiquetas de clase
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir etiquetas a one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Crear el modelo de red neuronal
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)